## Download html files

request,bsでは動的コンテンツをとってこれないのでseleniumを使う

In [2]:
import requests

In [3]:
url="https://toho-vote.info/result"

In [4]:
r=requests.get(url)

In [5]:
r

<Response [200]>

In [ ]:
r.content

In [ ]:
r.write()

## htmlからcsvファイル作成(scraping)

In [4]:
from bs4 import BeautifulSoup  # BeautifulSoupクラスをインポート

In [5]:
ddir="data"
filename_tr="/18/第18回東方Project人気投票_人妖"
filename=ddir+filename_tr+".html"

In [5]:
with open(ddir+filename,"r") as fp:
    htmltext=fp.read()
soup = BeautifulSoup(htmltext, "html.parser")

ブラウザのデバッガ
document.querySelector("#app > div > main > div._container_1r0po_1._root_1orw9_14 > div > div > div > table")
の結果から

In [4]:
soup.body.div.div.main.find("div", class_="_root_1orw9_14").div.div

<div class="_title_en_1h8e9_51">Characters</div>

In [5]:
soup.body.div.div.main.find("div", class_="_root_1orw9_14").div.header

<header class="_title_1h8e9_1"><h1>人妖部門</h1><div class="_title_en_1h8e9_51">Characters</div></header>

In [6]:
def get_tabledata(table):
    return [[td.string for td in tr.find_all("td")] for tr in table]

In [7]:
table=soup.body.div.div.main.find("div", class_="_root_1orw9_14").div


In [8]:
table_header=table.find_all("th")
table_contents=table.find_all("tr")

In [9]:
table_header

[<th>順位</th>,
 <th class="_rank_prev_1orw9_133">前回</th>,
 <th class="_rank_prev2_1orw9_136">前々</th>,
 <th class="_name_1orw9_101">名前</th>,
 <th>ポイント</th>,
 <th>一推し</th>,
 <th>コメント</th>,
 <th>支援作品</th>]

In [8]:
def get_tabledata(table):
    return [[td.string for td in tr.find_all("td")] for tr in table]    
def get_tableheader(table):
    return [[t.string for t in tr.find_all("th")] for tr in table]    

In [12]:
outdata=get_tabledata(table_contents)

In [14]:
outdata[0]=outheader
outdata[:2]

[[], ['1', '5', '5', 'フランドール・スカーレット', '16,723', '3,730', '3,633', '19']]

### output

## 第16回以前の投票結果

文字コード

In [85]:
!nkf -g */*/*.html

data/1/東方シリーズキャラ人気投票 [Vote Top].html: CP932
data/1/東方シリーズ音楽人気投票 [Vote Top].html: CP932
data/1/東方シリーズ音楽人気投票 [Vote Top]_iro.html: CP932
data/10/第１０回東方シリーズ人気投票 キャラ部門集計結果.html: EUC-JP
data/10/第１０回東方シリーズ人気投票 音楽部門集計結果.html: EUC-JP
data/10/第１０回東方シリーズ人気投票 投票者アンケート結果.html: EUC-JP
data/11/アンケート投票結果 - 第11回 東方Project 人気投票.html: UTF-8
data/11/キャラ部門投票結果 - 第11回 東方Project 人気投票.html: UTF-8
data/11/音楽部門投票結果 - 第11回 東方Project 人気投票.html: UTF-8
data/11/経緯報告書 - 第11回 東方Project 人気投票.html: UTF-8
data/12/アンケート部門集計結果 - 第12回 東方Project 人気投票.html: UTF-8
data/12/音楽部門投票結果 - 第12回 東方Project 人気投票.html: UTF-8
data/12/作品部門投票結果 - 第12回 東方Project 人気投票.html: UTF-8
data/12/人妖部門投票結果 - 第12回 東方Project 人気投票.html: UTF-8
data/13/アンケート集計結果 - 第13回 東方Project 人気投票.html: UTF-8
data/13/ベストパートナー部門投票結果 - 第13回 東方Project 人気投票.html: UTF-8
data/13/音楽部門投票結果 - 第13回 東方Project 人気投票.html: UTF-8
data/13/作品部門投票結果 - 第13回 東方Project 人気投票.html: UTF-8
data/13/人妖部門投票結果 - 第13回 東方Project 人気投票.html: UTF-8
data/14/アンケート部門集計結果 - 第14回 東方Project 人気投票.html: UTF-8
da

In [21]:
import csv
def openhtml(ddir,filename_tr):
    filename=ddir+filename_tr+".html"
    try:
        with open(filename,"r") as fp:
            htmltext=fp.read()
    except:
        try:
            with open(filename,"r",encoding='euc_jp') as fp:
                htmltext=fp.read()
        except:
            with open(filename,"r",encoding='CP932') as fp:
                htmltext=fp.read()
            print(filename)
    return htmltext
    
def souptable2csv(table,writedown=False,filename_tr=""):
    table_header=table.find_all("th")
    table_contents=table.find_all("tr")

    outdata  =get_tabledata(table_contents)
    outheader=get_tableheader(table_header)
#    outdata[0]=outheader[0]

    with open(ddir+filename_tr+".csv", 'w') as fpw:
        writer = csv.writer(fpw, lineterminator='\n')
        writer.writerows(outdata)

def converthtml2csv(filename_tr,writedown=False):
    htmltext=openhtml(ddir,filename_tr)
    soup = BeautifulSoup(htmltext, "html.parser")
#    table=soup.body.div.div.main.find("div", class_="_root_1orw9_14").div
    table=soup
    souptable2csv(table,writedown,filename_tr)

In [66]:
filename_tr="/18/第18回東方Project人気投票_人妖"
converthtml2csv(filename_tr)

In [67]:
filename_tr="/17/人妖"
converthtml2csv(filename_tr)

In [69]:
filename_tr="/16/人妖部門投票結果 - 第16回 東方Project 人気投票"
converthtml2csv(filename_tr)

In [22]:
import glob
htmlfiles=glob.glob("data/*/*.html")

for f in htmlfiles:
    filename_tr=f[4:]
    filename_tr=filename_tr.split(".")[0]
    #print (filename_tr)
    converthtml2csv(filename_tr)

data/4/第４回東方シリーズ人気投票 投票者アンケート結果.html
data/4/第４回東方シリーズ人気投票 その他集計結果.html
data/4/第４回東方シリーズ人気投票 キャラ部門集計結果.html
data/4/第４回東方シリーズ人気投票 音楽部門集計結果.html
data/3/第３回東方シリーズ人気投票 その他集計結果.html
data/3/第３回東方シリーズ人気投票 音楽部門集計結果.html
data/3/第３回東方シリーズ人気投票 キャラ部門集計結果.html
data/3/第３回東方シリーズ人気投票 スペルカード部門集計結果.html
data/3/第３回東方シリーズ人気投票 投票者アンケート結果.html
data/2/第２回東方シリーズ人気投票 音楽部門結果.html
data/2/第２回東方シリーズ人気投票 キャラ部門結果.html
data/2/第２回東方シリーズ人気投票 その他集計結果.html
data/2/第２回東方シリーズ人気投票 弾幕部門結果.html
data/1/東方シリーズ音楽人気投票 [Vote Top]_iro.html
data/1/東方シリーズキャラ人気投票 [Vote Top].html
data/1/東方シリーズ音楽人気投票 [Vote Top].html


In [23]:
!wc data/*/*.csv

    76     55    593 data/1/東方シリーズキャラ人気投票 [Vote Top].csv
   225    237   2480 data/1/東方シリーズ音楽人気投票 [Vote Top].csv
   225    237   2480 data/1/東方シリーズ音楽人気投票 [Vote Top]_iro.csv
   160    225   8124 data/10/第１０回東方シリーズ人気投票 キャラ部門集計結果.csv
   432    823  27956 data/10/第１０回東方シリーズ人気投票 音楽部門集計結果.csv
   216    228   5792 data/10/第１０回東方シリーズ人気投票 投票者アンケート結果.csv
   268    264   5883 data/11/アンケート投票結果 - 第11回 東方Project 人気投票.csv
   173    239   8701 data/11/キャラ部門投票結果 - 第11回 東方Project 人気投票.csv
   916   1308  26604 data/11/音楽部門投票結果 - 第11回 東方Project 人気投票.csv
     0      0      0 data/11/経緯報告書 - 第11回 東方Project 人気投票.csv
   260    257   5976 data/12/アンケート部門集計結果 - 第12回 東方Project 人気投票.csv
   476    858  27791 data/12/音楽部門投票結果 - 第12回 東方Project 人気投票.csv
    42     47   1888 data/12/作品部門投票結果 - 第12回 東方Project 人気投票.csv
   170    237   9271 data/12/人妖部門投票結果 - 第12回 東方Project 人気投票.csv
   274    271   6368 data/13/アンケート集計結果 - 第13回 東方Project 人気投票.csv
   438   1002  24687 data/13/ベストパートナー部門投票結果 - 第13回 東方Project 人気投票.csv
   4

In [74]:
htmlfiles

['data/spell/スペルカード人気投票 使用者別集計結果.html',
 'data/spell/スペルカード人気投票 集計結果.html',
 'data/spell/アンケート・文花帖クリアシーン数結果.html',
 'data/spell/スペルカード人気投票 系統別集計結果.html',
 'data/12/音楽部門投票結果 - 第12回 東方Project 人気投票.html',
 'data/12/作品部門投票結果 - 第12回 東方Project 人気投票.html',
 'data/12/人妖部門投票結果 - 第12回 東方Project 人気投票.html',
 'data/12/アンケート部門集計結果 - 第12回 東方Project 人気投票.html',
 'data/16/作品部門投票結果 - 第16回 東方Project 人気投票.html',
 'data/16/人妖部門投票結果 - 第16回 東方Project 人気投票.html',
 'data/16/アンケート部門集計結果 - 第16回 東方Project 人気投票.html',
 'data/16/音楽部門投票結果 - 第16回 東方Project 人気投票.html',
 'data/16ex/ベストパートナー部門投票結果 - 第16回 東方Project 人気投票Ex.html',
 'data/10/第１０回東方シリーズ人気投票 投票者アンケート結果.html',
 'data/10/第１０回東方シリーズ人気投票 キャラ部門集計結果.html',
 'data/10/第１０回東方シリーズ人気投票 音楽部門集計結果.html',
 'data/5/第５回東方シリーズ人気投票 投票者アンケート結果.html',
 'data/5/第５回東方シリーズ人気投票 音楽部門集計結果.html',
 'data/5/第５回東方シリーズ人気投票 その他集計結果.html',
 'data/5/第５回東方シリーズ人気投票 キャラ部門集計結果.html',
 'data/13/作品部門投票結果 - 第13回 東方Project 人気投票.html',
 'data/13/アンケート集計結果 - 第13回 東方Project 人気投票.html',
 'data/13/音楽部門投票結